In [1]:
import requests
import pandas as pd
import re
import numpy as np
from bs4 import BeautifulSoup

In [67]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}

response = requests.get('http://www.metrolyrics.com/king-krule-lyrics.html', headers=headers)
doc = BeautifulSoup(response.text)

In [68]:
table = doc.find_all(class_="songs-table compact")[0]
songs = table.find_all("tr")[1:]

In [69]:
rows = []
for song in songs: 
    row = {}
    row['title'] = song.find(class_="title").text.strip()
    row['url'] = song.find('a')['href']
    row['popularity'] = song.find(class_='bar')
    row['year'] = song.find_all('td')[2].text
    rows.append(row)

df = pd.DataFrame(rows)

In [70]:
df.head()

,popularity,title,url,year
0,"<span class=""bar popular10""> <span style=""widt...",Baby Blue Lyrics,http://www.metrolyrics.com/baby-blue-lyrics-ki...,2015
1,"<span class=""bar popular5""> <span style=""width...",A Lizard State Lyrics,http://www.metrolyrics.com/a-lizard-state-lyri...,2014
2,"<span class=""bar popular5""> <span style=""width...",Easy Easy Lyrics,http://www.metrolyrics.com/easy-easy-lyrics-ki...,2013
3,"<span class=""bar popular4""> <span style=""width...",Out Getting Ribs Lyrics,http://www.metrolyrics.com/out-getting-ribs-ly...,2014
4,"<span class=""bar popular2""> <span style=""width...",Dum Surfer Lyrics,http://www.metrolyrics.com/dum-surfer-lyrics-k...,2017


In [71]:
# df['score'] = df.score.str.replace(",","")
# df['score'] = df.score.str.extract(r"score: (.*)")

In [72]:
df['popularity'] = df.popularity.astype(str)

In [73]:
df['popularity'] = df.popularity.str.extract(r"popular(\d\d?)")

In [74]:
df['popularity'] = df.popularity.astype(int)

In [75]:
df.head()

,popularity,title,url,year
0,10,Baby Blue Lyrics,http://www.metrolyrics.com/baby-blue-lyrics-ki...,2015
1,5,A Lizard State Lyrics,http://www.metrolyrics.com/a-lizard-state-lyri...,2014
2,5,Easy Easy Lyrics,http://www.metrolyrics.com/easy-easy-lyrics-ki...,2013
3,4,Out Getting Ribs Lyrics,http://www.metrolyrics.com/out-getting-ribs-ly...,2014
4,2,Dum Surfer Lyrics,http://www.metrolyrics.com/dum-surfer-lyrics-k...,2017


In [76]:
df.to_csv("king_krule.csv",index=False)

In [77]:
df = pd.read_csv("king_krule.csv")

In [85]:
url = "http://www.metrolyrics.com/baby-blue-lyrics-king-krule.html"
print("We are scraping", url)
response = requests.get(url)
doc = BeautifulSoup(response.text)

verses = doc.find_all(class_="verse")
for verse in verses:
    print(verse.text.strip())

We are scraping http://www.metrolyrics.com/baby-blue-lyrics-king-krule.html
My sandpaper sigh engraves a line
Into the rust of your tongue
Girl I could've been someone, to you
Would have painted the skies blue
Baby blue
If you knew
Baby blue
Baby blue
Baby blue
Edging closer, you swing my way
Girl I've got no chance
And nothing to say
Girl but stay here for a little while
Baby blue
Baby blue
Baby blue
But if only you could see
My shadow crossing your path
It won't be the last baby blue
You are my baby
You are my baby
Bobbing on, the problem
On the problem, on the throbbing
She was falling, I was bawling
But I was stopping, she was throbbing


In [79]:
def scrape_page(row):
    url = f"{row['url']}"
    print("Scraping", row['url'])

    response = requests.get(url)
    doc = BeautifulSoup(response.text)

    page = {}

    verses = doc.find_all(class_="verse")
    for verse in verses:
        page['lyrics'] = verse.text.strip()

    return pd.Series(page)

In [80]:
scraped_df = df.apply(scrape_page, axis=1)

Scraping http://www.metrolyrics.com/baby-blue-lyrics-king-krule.html
Scraping http://www.metrolyrics.com/baby-blue-lyrics-king-krule.html
Scraping http://www.metrolyrics.com/a-lizard-state-lyrics-king-krule.html
Scraping http://www.metrolyrics.com/easy-easy-lyrics-king-krule.html
Scraping http://www.metrolyrics.com/out-getting-ribs-lyrics-king-krule.html
Scraping http://www.metrolyrics.com/dum-surfer-lyrics-king-krule.html
Scraping http://www.metrolyrics.com/portrait-in-black-and-blue-lyrics-king-krule.html
Scraping http://www.metrolyrics.com/the-noose-of-jah-city-lyrics-king-krule.html
Scraping http://www.metrolyrics.com/bleak-bake-lyrics-king-krule.html
Scraping http://www.metrolyrics.com/border-line-lyrics-king-krule.html
Scraping http://www.metrolyrics.com/lead-existence-lyrics-king-krule.html
Scraping http://www.metrolyrics.com/little-wild-lyrics-king-krule.html
Scraping http://www.metrolyrics.com/intro-lyrics-king-krule.html
Scraping http://www.metrolyrics.com/neptune-estate-lyri

In [91]:
scraped_df.lyrics

0     Bobbing on, the problem\nOn the problem, on th...
1     I don't care about sunny days\ngonna keep it o...
2     You're easy\nSo easy\nYou're easy\nMan just le...
3     But wait\nI make my last request\nYours could ...
4     So keep me as the villain\nBut my prayer, you ...
5     I ain't painting this lie any more since it wa...
6     It eats away at your brain\nAs you strain to t...
7     And everything hits you in the end\nAnd spoils...
8     But my soul floats\nAdrift thoughtless minds, ...
9     An empty room is now my doom\nMy past was gone...
10    (bonus track featured on the Japanese edition ...
11                                                  NaN
12    I could stay in your mind\nWish me to stray aw...
13    Dumudulas sa dumi\nMalungkot pero napapalibuta...
14    We all have our evils\nWe're told just to keep...
15                                                  NaN
16    Deslizando entre suciedad a solas pero rodeado...
17    Yeah no help still\nNo help still\nNo help

In [92]:
final_df = df.merge(scraped_df, left_index=True, right_index=True)

In [93]:
final_df.to_csv("scraped_king_krule.csv",index=False)